In [ ]:
import pandas as pd

# Check the available columns
# Step 1: Load the dataset
file_name = "supply_chain_data.csv"
data = pd.read_csv(file_name)
print(data.columns)


Index(['Product type', 'SKU', 'Price', 'Availability',
       'Number of products sold', 'Revenue generated', 'Customer demographics',
       'Stock levels', 'Lead times', 'Order quantities', 'Shipping times',
       'Shipping carriers', 'Shipping costs', 'Supplier name', 'Location',
       'Lead time', 'Production volumes', 'Manufacturing lead time',
       'Manufacturing costs', 'Inspection results', 'Defect rates',
       'Transportation modes', 'Routes', 'Costs'],
      dtype='object')


In [ ]:
# 1. Extract Transportation Costs (C_ij)
transportation_costs = data[['Routes', 'Costs']].groupby('Routes').mean()
print(transportation_costs)

              Costs
Routes             
Route A  485.483128
Route B  595.659028
Route C  500.470985


In [ ]:
# 2. Extract Demand for Products (D_j)
demand = data.groupby('Product type')['Order quantities'].sum()
print(demand)

Product type
cosmetics    1343
haircare     1480
skincare     2099
Name: Order quantities, dtype: int64


In [ ]:
# 3. Extract Production Capacity (S_i)
production_capacity = data.groupby('Routes')['Production volumes'].sum()
print(production_capacity)

Routes
Route A    24250
Route B    20039
Route C    12495
Name: Production volumes, dtype: int64


In [ ]:
# 5. Verify Data
print(f"Transportation Costs:\n{transportation_costs}")
print(f"Demand:\n{demand}")
print(f"Production Capacity:\n{production_capacity}")

# 6. Save Relevant Data
# Save for use in optimization
transportation_costs.to_csv('transportation_costs.csv')
demand.to_csv('demand.csv')
production_capacity.to_csv('production_capacity.csv')


Transportation Costs:
              Costs
Routes             
Route A  485.483128
Route B  595.659028
Route C  500.470985
Demand:
Product type
cosmetics    1343
haircare     1480
skincare     2099
Name: Order quantities, dtype: int64
Production Capacity:
Routes
Route A    24250
Route B    20039
Route C    12495
Name: Production volumes, dtype: int64


In [ ]:
from pulp import LpProblem, LpMinimize, LpVariable, lpSum, value
import pandas as pd


In [ ]:
# Routes and their production capacities
production_capacity = {
    "Route A": 24250,
    "Route B": 20039,
    "Route C": 12495
}

# Products and their demand
demand = {
    "cosmetics": 1343,
    "haircare": 1480,
    "skincare": 2099
}

# Transportation costs
costs = {
    ("Route A", "cosmetics"): 485.48,
    ("Route A", "haircare"): 485.48,
    ("Route A", "skincare"): 485.48,
    ("Route B", "cosmetics"): 595.66,
    ("Route B", "haircare"): 595.66,
    ("Route B", "skincare"): 595.66,
    ("Route C", "cosmetics"): 500.47,
    ("Route C", "haircare"): 500.47,
    ("Route C", "skincare"): 500.47
}

NameError: name 'costs_df' is not defined

In [ ]:
# Create a DataFrame for production capacity
prod_capacity_df = pd.DataFrame(list(production_capacity.items()), columns=["Route", "Production Capacity"])
print(prod_capacity_df)


     Route  Production Capacity
0  Route A                24250
1  Route B                20039
2  Route C                12495


In [ ]:
# Create a DataFrame for demand
demand_df = pd.DataFrame(list(demand.items()), columns=["Product", "Demand"])
print(demand_df)


     Product  Demand
0  cosmetics    1343
1   haircare    1480
2   skincare    2099


In [ ]:
# Create DataFrame for costs
costs_df = pd.DataFrame(list(costs.items()), columns=["Route & Product", "Cost"])
costs_df[["Route", "Product"]] = pd.DataFrame(costs_df["Route & Product"].tolist(), index=costs_df.index)
costs_df = costs_df.drop(columns=["Route & Product"])
costs_df = costs_df.pivot(index="Route", columns="Product", values="Cost").reset_index()
print(costs_df)


Product    Route  cosmetics  haircare  skincare
0        Route A     485.48    485.48    485.48
1        Route B     595.66    595.66    595.66
2        Route C     500.47    500.47    500.47


## ***Decision***

In [ ]:
# Create the optimization model
model = LpProblem("Supply_Chain_Optimization", LpMinimize)

# Decision variables: Quantity of each product transported via each route (as integers)
X = LpVariable.dicts(
    "Transport",
    [(route, product) for route in production_capacity.keys() for product in demand.keys()],
    lowBound=0,
    cat='Integer'  # Change from 'Continuous' to 'Integer'
)


## ***Objective***

In [ ]:
# Objective function: Minimize total transportation cost
model += lpSum([costs[(route, product)] * X[(route, product)] for route, product in X])


## ***Constraints***

#### ***Demand Satisfaction***

In [ ]:
for product in demand.keys():
    model += lpSum([X[(route, product)] for route in production_capacity.keys()]) >= demand[product], f"Demand_{product}"


#### ***Production Capacity***

In [ ]:
for route in production_capacity.keys():
    model += lpSum([X[(route, product)] for product in demand.keys()]) <= production_capacity[route], f"Capacity_{route}"


#### ***Fair Distribution***

In [ ]:
# Calculate the total demand
total_demand = sum(demand.values())

# Add fair distribution constraint: Each route must handle at least 10% of the total demand
fair_share = 0.1  # Minimum proportion of total demand for each route
for route in production_capacity.keys():
    model += lpSum([X[(route, product)] for product in demand.keys()]) >= fair_share * total_demand, f"Fair_Share_{route}"


## ***Optimize Solution***

In [ ]:
# Solve the problem
model.solve()

# Output results
for var in X:
    print(f"{var}: {X[var].value()} tons transported.")
print(f"Total Cost: ${model.objective.value()}")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.12/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/mh/9kjbqf2n1g9986vkzqxgmc1h0000gn/T/cabd2b92f55f42c2bbdf47ac9f696279-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/mh/9kjbqf2n1g9986vkzqxgmc1h0000gn/T/cabd2b92f55f42c2bbdf47ac9f696279-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 14 COLUMNS
At line 69 RHS
At line 79 BOUNDS
At line 89 ENDATA
Problem MODEL has 9 rows, 9 columns and 27 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 2.45114e+06 - 0.00 seconds
Cgl0004I processed model has 6 rows, 9 columns (9 integer (0 of which binary)) and 18 elements
Cutoff increment increased from 1e-05 to 0.00999
Cbc0012I Integer solution of 2452212.3 found by DiveCoefficient after 0 iterations and 0 nodes (0.01 seconds)
Cbc0038I Full p